In [1]:
import pandas as pd
import re

In [2]:
path_to_csv = "Peru_2019_AudioMoth_Data_Full.csv"
df_audio = pd.read_csv(path_to_csv, dtype={'AudioMothID': 'str', 'StartDateTime':'str', 'Artist':'str'})
df_audio.shape[0]

98638

In [3]:
# filter to clips that are at least 60 seconds long and did not have errors
df_audio = df_audio[(df_audio['Duration'] >= 60) & 
                    (df_audio['Error'].isnull()) &
                    (df_audio['FileSize'] >= 46080000)]
df_audio.shape[0]

93690

In [4]:
# if startdatetime is na get it from comments
df_audio['backupTime'] = df_audio['Comment'].apply(lambda x : re.search(r'\d{2}:\d{2}:\d{2} \d{2}/\d{2}/\d{4}', x).group())
df_audio['StartDateTime'].fillna(df_audio['backupTime'], inplace=True)

In [5]:
# create a new column called hour, gets hour from the start date
df_audio['hour'] = pd.to_datetime(df_audio['StartDateTime']).dt.hour

# for each unique audiomoth, remove those that are smaller than 24 hours
for audiomoth in df_audio["AudioMothCode"].unique():
    time_df = df_audio[df_audio['AudioMothCode']==audiomoth]
    if not (time_df['hour'].unique().size == 24):
        df_audio = df_audio[df_audio['AudioMothCode'] != audiomoth]
print(df_audio.shape[0])

# sample one from each hour
temp_df = df_audio.groupby(['AudioMothCode', 'hour']).apply(lambda x: x.sample(1))
print(temp_df.shape[0])
#export to csv
temp_df.to_csv('stratified.csv')

93682
744


Program does
- take a csv file
- checks that a clip from each hour of the day exists, removes those that have less that that
- randomly selects a clip from each of the 24 hours for each audiomoth
- turns into csv file
- use comments to fix the WWF data
- make it a function

TODO
- why did audiomoth 21, 19, 8, 28 have problems? i think only 18 had an issue in the given dataset


